In [2]:
from utils.kusto_connection import kc
import pandas as pd

In [42]:
q = '''PREDISPATCHPRICE_ADF
| where LASTCHANGED between (startofday(datetime('2025-08-04')) .. endofday(datetime('2025-08-04')) )
| where REGIONID == 'NSW1' 
| project LASTCHANGED,DATETIME,RRP
| extend interval_end = DATETIME
| extend time_of_run = bin(LASTCHANGED, 30m)
| summarize arg_max(LASTCHANGED, *) by interval_end, time_of_run
| where interval_end > time_of_run
| project interval_end, time_of_run, RRP
| extend prio = 0, interval_start = interval_end - 30m;
//
P5MIN_REGIONSOLUTION
| where RUN_DATETIME between (startofday(datetime('2025-08-04')) .. endofday(datetime('2025-08-04')) )
| where REGIONID == 'NSW1'
| project RUN_DATETIME, INTERVAL_DATETIME, RRP
| extend interval_end = INTERVAL_DATETIME
| extend time_of_run = bin(RUN_DATETIME,5m)
| summarize arg_max(RUN_DATETIME, *) by interval_end, time_of_run
| where interval_end > time_of_run
| project interval_end, time_of_run, RRP
| extend prio = 1, interval_start = interval_end - 5m;'''

pre,p5 =  kc('btmoinfoserver', q)
def drop_tz(t):
    return t.tz_convert(None)
pre.interval_start = pre.interval_start.apply(drop_tz)
pre.interval_end = pre.interval_end.apply(drop_tz)
pre.time_of_run = pre.time_of_run.apply(drop_tz)
p5.interval_start = p5.interval_start.apply(drop_tz)
p5.interval_end = p5.interval_end.apply(drop_tz)
p5.time_of_run = p5.time_of_run.apply(drop_tz)

pre = pre[['time_of_run', 'interval_start','interval_end', 'RRP', 'prio']]
p5= p5[['time_of_run', 'interval_start','interval_end', 'RRP', 'prio']]

In [45]:
run_start_5 = pd.Timestamp('2025-08-04 10:10:00').round('5min')
run_start_30 = pd.Timestamp('2025-08-04 10:10:00').floor('30min')

In [65]:
current_pre = pre[pre.time_of_run == run_start_30].reset_index(drop=True)
current_p5 = p5[p5.time_of_run == run_start_5].reset_index(drop=True)
current_pre = current_pre[current_pre.interval_end > current_p5.loc[current_p5.shape[0]-1,'interval_end']].reset_index(drop=True)
current_pre.loc[0,'interval_start'] = current_p5.loc[current_p5.shape[0]-1, 'interval_end']

current = pd.concat([current_p5,current_pre]).reset_index()
current['duration'] = (current.interval_end - current.interval_start)
current.duration = current.duration.apply(lambda x: x.seconds)

In [67]:
current

,index,time_of_run,interval_start,interval_end,RRP,prio,duration
0,0,2025-08-04 10:10:00,2025-08-04 10:10:00,2025-08-04 10:15:00,0.06,1,300
1,1,2025-08-04 10:10:00,2025-08-04 10:15:00,2025-08-04 10:20:00,-7.27763,1,300
2,2,2025-08-04 10:10:00,2025-08-04 10:20:00,2025-08-04 10:25:00,-7.27763,1,300
3,3,2025-08-04 10:10:00,2025-08-04 10:25:00,2025-08-04 10:30:00,-9.5,1,300
4,4,2025-08-04 10:10:00,2025-08-04 10:30:00,2025-08-04 10:35:00,-7.22633,1,300
5,5,2025-08-04 10:10:00,2025-08-04 10:35:00,2025-08-04 10:40:00,0.06,1,300
6,6,2025-08-04 10:10:00,2025-08-04 10:40:00,2025-08-04 10:45:00,-7.13248,1,300
7,7,2025-08-04 10:10:00,2025-08-04 10:45:00,2025-08-04 10:50:00,-7.13248,1,300
8,8,2025-08-04 10:10:00,2025-08-04 10:50:00,2025-08-04 10:55:00,-7.78022,1,300
9,9,2025-08-04 10:10:00,2025-08-04 10:55:00,2025-08-04 11:00:00,-9.17918,1,300


In [41]:
pre[['time_of_run', 'interval_start','interval_end', 'RRP', 'prio']]

,time_of_run,interval_start,interval_end,RRP,prio
0,2025-08-04 23:30:00,2025-08-04 23:30:00,2025-08-05 00:00:00,108.89,0
1,2025-08-04 23:30:00,2025-08-05 00:00:00,2025-08-05 00:30:00,104.53316,0
2,2025-08-04 23:30:00,2025-08-05 00:30:00,2025-08-05 01:00:00,93.15792,0
3,2025-08-04 23:30:00,2025-08-05 01:00:00,2025-08-05 01:30:00,89.34574,0
4,2025-08-04 23:30:00,2025-08-05 01:30:00,2025-08-05 02:00:00,84.79,0
...,...,...,...,...,...
2659,2025-08-04 13:00:00,2025-08-06 01:30:00,2025-08-06 02:00:00,72.78001,0
2660,2025-08-04 13:00:00,2025-08-06 02:00:00,2025-08-06 02:30:00,71.58322,0
2661,2025-08-04 13:00:00,2025-08-06 02:30:00,2025-08-06 03:00:00,75.49418,0
2662,2025-08-04 13:00:00,2025-08-06 03:00:00,2025-08-06 03:30:00,75.04529,0


In [26]:
month,cats = [],[]
v_cats = ['volitile', 'variable_but_below_strike', 'flatter']
## get most vol months 
def get_months_for_cat(cat,val, m,c, n=2, vol = False):
    if vol:
        if not isinstance(n,list):
            n = [n for i in v_cats]
        for i,v in enumerate(v_cats):
            if n[i] == 0:
                continue
            temp = prices_agg_agg[prices_agg_agg.volitility == v].groupby('month')[[val]].sum().sort_values(val, ascending=False).reset_index()
            m.extend(temp.iloc[:n[i]].month.to_list())
            c.extend([v+' '+cat]*n[i])
    else:
        temp = prices_agg_agg.groupby('month')[[val]].sum().sort_values(val, ascending=False).reset_index()
        m.extend(temp.iloc[:n].month.to_list())
        c.extend([cat]*n)
    return m,c

cat = 'highest high price months'
val = 'dp_above_1000_score'
month, cats = get_months_for_cat(cat,val,month,cats)

cat = 'highest abs price months'
val = 'absolute_dp_score'
month, cats = get_months_for_cat(cat,val,month,cats)

cat = 'highest spread months'
val = 'spread'
month, cats = get_months_for_cat(cat,val,month,cats)

## High low spread
cat = 'high spread'
val = 'spread_high'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'low spread'
val = 'spread_low'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

## RMSE ACC
cat = 'accurate p5 rmse'
val = 'rmse_p5_accurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'inaccurate p5 rmse'
val = 'rmse_p5_inaccurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'accurate pd rmse'
val = 'rmse_pd_accurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'inaccurate pd rmse'
val = 'rmse_pd_inaccurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

## SPREAD ACC
cat = 'accurate p5 spread period'
val = 'max_period_score_p5_accurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'inaccurate p5 spread period'
val = 'max_period_score_p5_inaccurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'accurate pd spread period'
val = 'max_period_score_pd_accurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'inaccurate pd spread period'
val = 'max_period_score_pd_inaccurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

df = pd.DataFrame({'Month': month, 'Categories':cats}).groupby('Month').agg(lambda x: list(x)).reset_index()


,Month,Categories
0,2021-11-01,"[volitile low spread, volitile accurate p5 rms..."
1,2022-01-01,"[flatter low spread, flatter accurate p5 rmse,..."
2,2022-04-01,"[variable_but_below_strike low spread, variabl..."
3,2022-05-01,[variable_but_below_strike inaccurate pd rmse]
4,2022-06-01,"[highest abs price months, variable_but_below_..."
5,2022-07-01,"[highest abs price months, variable_but_below_..."
6,2022-10-01,"[variable_but_below_strike accurate p5 rmse, v..."
7,2022-12-01,"[flatter inaccurate p5 spread period, flatter ..."
8,2023-03-01,[volitile accurate pd rmse]
9,2023-04-01,[variable_but_below_strike accurate p5 spread ...


In [33]:
prices_agg_agg.to_clipboard(index = None)

In [24]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.helpers import dataframe_from_result_table
from enum import StrEnum


class Clusters(StrEnum): 
    prod = "https://btmorchestrationprod.australiasoutheast.kusto.windows.net/"
    shared = "https://btmorchestrationshared.australiasoutheast.kusto.windows.net"
    dev = "https://neoadxdev.australiasoutheast.kusto.windows.net"

class Kusto_Connection():
    def __init__(self, cluster = Clusters.shared):
        self.cluster = cluster
        
        builder = KustoConnectionStringBuilder.with_interactive_login(cluster)
        self.kusto_client = KustoClient(builder)

    def query(self, db, query):
        response = self.kusto_client.execute(db, query)
        out = [dataframe_from_result_table(r) for r in response.primary_results]
        return out[0] if len(out) == 1 else out

    def __call__(self, db, query):
        return self.query(db=db,query=query)
    
kc = Kusto_Connection()

In [44]:
q = """PREDISPATCHPRICE_ADF
| where LASTCHANGED between (startofday(datetime('2025-08-04')) .. endofday(datetime('2025-08-04')) )
| where REGIONID == 'NSW1' 
| project LASTCHANGED,DATETIME,RRP
| extend interval_end = DATETIME
| extend time_of_run = bin(LASTCHANGED, 30m)
| summarize arg_max(LASTCHANGED, *) by interval_end, time_of_run
| where interval_end > time_of_run
| extend prio = 0, interval_start = interval_end - 30m
| project time_of_run, interval_start, interval_end, RRP
| order by time_of_run asc, interval_start asc;
P5MIN_REGIONSOLUTION
| where RUN_DATETIME between (startofday(datetime('2025-08-04')) .. endofday(datetime('2025-08-04')) )
| where REGIONID == 'NSW1'
| project RUN_DATETIME, INTERVAL_DATETIME, RRP
| extend interval_end = INTERVAL_DATETIME
| extend time_of_run = bin(RUN_DATETIME,5m)
| summarize arg_max(RUN_DATETIME, *) by interval_end, time_of_run
| where interval_end > time_of_run
| extend prio = 1, interval_start = interval_end - 5m
| project time_of_run, interval_start, interval_end, RRP
| order by time_of_run asc, interval_start asc;"""


prd,p5 =  kc('btmoinfoserver',q)

In [41]:
prd.time_of_run = prd.time_of_run.a

,time_of_run,interval_start,interval_end,RRP
0,2025-08-04 00:00:00+00:00,2025-08-04 00:00:00+00:00,2025-08-04 00:30:00+00:00,80.02
1,2025-08-04 00:00:00+00:00,2025-08-04 00:30:00+00:00,2025-08-04 01:00:00+00:00,76.8
2,2025-08-04 00:00:00+00:00,2025-08-04 01:00:00+00:00,2025-08-04 01:30:00+00:00,72.47781
3,2025-08-04 00:00:00+00:00,2025-08-04 01:30:00+00:00,2025-08-04 02:00:00+00:00,64.89
4,2025-08-04 00:00:00+00:00,2025-08-04 02:00:00+00:00,2025-08-04 02:30:00+00:00,57.01
...,...,...,...,...
2659,2025-08-04 23:30:00+00:00,2025-08-06 01:30:00+00:00,2025-08-06 02:00:00+00:00,76.58
2660,2025-08-04 23:30:00+00:00,2025-08-06 02:00:00+00:00,2025-08-06 02:30:00+00:00,75.38122
2661,2025-08-04 23:30:00+00:00,2025-08-06 02:30:00+00:00,2025-08-06 03:00:00+00:00,76.58
2662,2025-08-04 23:30:00+00:00,2025-08-06 03:00:00+00:00,2025-08-06 03:30:00+00:00,76.80013


In [42]:
p5

,time_of_run,interval_start,interval_end,RRP
0,2025-08-04 00:00:00+00:00,2025-08-04 00:00:00+00:00,2025-08-04 00:05:00+00:00,83.74338
1,2025-08-04 00:00:00+00:00,2025-08-04 00:05:00+00:00,2025-08-04 00:10:00+00:00,84.26909
2,2025-08-04 00:00:00+00:00,2025-08-04 00:10:00+00:00,2025-08-04 00:15:00+00:00,84.85
3,2025-08-04 00:00:00+00:00,2025-08-04 00:15:00+00:00,2025-08-04 00:20:00+00:00,84.26909
4,2025-08-04 00:00:00+00:00,2025-08-04 00:20:00+00:00,2025-08-04 00:25:00+00:00,80.65875
...,...,...,...,...
3163,2025-08-04 23:55:00+00:00,2025-08-05 00:25:00+00:00,2025-08-05 00:30:00+00:00,85.58303
3164,2025-08-04 23:55:00+00:00,2025-08-05 00:30:00+00:00,2025-08-05 00:35:00+00:00,87.50173
3165,2025-08-04 23:55:00+00:00,2025-08-05 00:35:00+00:00,2025-08-05 00:40:00+00:00,85.4976
3166,2025-08-04 23:55:00+00:00,2025-08-05 00:40:00+00:00,2025-08-05 00:45:00+00:00,85.58303
